In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests 
from urllib.parse import urlencode 

%matplotlib inline 


In [2]:
# используем api 
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
public_key = 'https://disk.yandex.ru/d/txn7q-CE2zSaPg' 
 
# получаем url 
final_url = base_url + urlencode(dict(public_key=public_key)) 
response = requests.get(final_url) 
download_url = response.json()['href'] 
 
# загружаем файл в df 
download_response = requests.get(download_url) 
dating_data = pd.read_csv(download_url) 

In [ ]:
# Пользователи в группе с номером 0 пользовались приложением со старым алгоритмом.
# Все пользователи в группе 1 пользовались приложением с новым алгоритмом для поиска анкет.

In [3]:
dating_data

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1
...,...,...,...,...
14509,476,631,0,0
14510,455,770,1,0
14511,260,6,1,1
14512,885,812,1,1


In [4]:
dating_data.group.value_counts()

1    9722
0    4792
Name: group, dtype: int64

In [10]:
dating_data.isna().sum()

user_id_1    0
user_id_2    0
group        0
is_match     0
dtype: int64

In [7]:
dating_data_0 = dating_data.query('group==0')
dating_data_0

,user_id_1,user_id_2,group,is_match
2,423,677,0,0
4,969,155,0,1
5,227,162,0,0
8,223,295,0,0
10,825,388,0,0
...,...,...,...,...
14502,535,545,0,0
14503,128,268,0,0
14505,544,600,0,0
14506,903,505,0,0


In [11]:
dating_data_1 = dating_data.query('group==1')
dating_data_1

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
3,658,165,1,1
6,435,16,1,0
7,602,327,1,0
...,...,...,...,...
14508,527,647,1,0
14510,455,770,1,0
14511,260,6,1,1
14512,885,812,1,1


In [27]:
dating_data_01 = dating_data_0.groupby('is_match' , as_index = False) \
                      .agg({'user_id_1':'count'}) \
                      .rename(columns = {'user_id_1':'counts'})

dating_data_01['percent_all'] = dating_data_01.counts.apply(lambda x: x/dating_data_01.counts.sum()*100)

In [28]:
dating_data_01

,is_match,counts,percent_all
0,0,3858,80.509182
1,1,934,19.490818


In [88]:
dating_data_11 = dating_data_1.groupby('is_match' , as_index = False) \
                      .agg({'user_id_1':'count'}) \
                      .rename(columns = {'user_id_1':'counts'})

dating_data_11['percent_all'] = dating_data_11.counts.apply(lambda x: x/dating_data_11.counts.sum()*100)

In [89]:
dating_data_11

,is_match,counts,percent_all
0,0,5813,59.792224
1,1,3909,40.207776


In [97]:
dating_data_1_all = dating_data_01.merge(dating_data_11, on = 'is_match')
dating_data_1_all = dating_data_1_all.rename(columns = {'counts_x':'counts_0g','percent_all_x':'percent_0g','counts_y':'counts_1g','percent_all_y':'percent_1g'})

dating_data_1_all_1 = dating_data_1_all[['counts_0g', 'counts_1g']]
dating_data_1_all_2 = dating_data_1_all[['is_match', 'percent_0g', 'percent_1g']]

In [98]:
dating_data_1_all_1

,counts_0g,counts_1g
0,3858,5813
1,934,3909


In [112]:
from scipy.stats import chi2_contingency
obs = dating_data_1_all_1
chi2_contingency(obs)

(618.6889141576197,
 1.441829916366341e-136,
 1,
 array([[3193.01584677, 6477.98415323],
        [1598.98415323, 3244.01584677]]))

In [113]:
# pvalue(1.4418299163662586e-136) ниже, чем 0,05, поэтому мы можем говорить о том, 
# что доля матчей в экспереметальной группе не соответвует аналогичному показателю из обычной группы.
# Дельта в % в мэтчах увеличилось на 20% в экспереметальной группе, поэтому предлагаем 
# накатить новый алгоритм поиска на всех пользователей.

In [114]:
dating_data_1_all_2['delta'] = dating_data_1_all_2['percent_1g']-dating_data_1_all_2['percent_0g']

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [107]:
dating_data_1_all_2

,is_match,percent_0g,percent_1g,delta
0,0,80.509182,59.792224,-20.716958
1,1,19.490818,40.207776,20.716958
